In [ ]:
import torch
import numpy as np
import torch.nn as nn

from torch.nn import functional as F

In [ ]:
torch.manual_seed(256)
device = 'cuda' if torch.cuda.is_available() else 'cpu' #Cuda is the drivers for Nvidia GPU

block_size        = 40      ## N tokens in sequence -> its not using tiktoken. Tokens are the individual letters
batch_size        = 64
max_iters         = 1       ## Lets try with 1 to learn
eval_interval     = 500     # Print what it is doing
learning_rate     = 0.0003  # Control how much we increase or decrease the weight values with the scocastic gradient descent
eval_iters        = 300     # Evaluate every this iterations
vocab_size        = 378      ## 65 -> 26 uppercase, 26 lowercase, digits, simbols...

## every id for a given token is embedded to vector of this size
n_embd            = 512
n_head            = 8         ## 8 attention heads
n_layer           = 6         ## 6 eoncoder layers
dropout           = 0.2       # We will se this later

In [ ]:
text = ''

#input_file2 = 'HuckFinn.txt'
input_file2 = '/content/drive/MyDrive/Colab Notebooks/dataFromRFCs.txt' # Full path

with open(input_file2, 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print(text[:1000])

2
3
5
20
31
33
57
59
66
86
88
89
90
124
127
162
194
270
304
542
586
640
675
761
783
787
791
792
793
804
819
822
823
826
854
864
865
867
868
879
882
889
894
916
917
918
951
952
957
959
994
1001
1002
1006
1025
1027
1033
1034
1035
1036
1042
1055
1065
1071
1122
1123
1138
1141
1148
1149
1150
1155
1157
1158
1172
1180
1183
1185
1195
1227
1258
1288
1319
1320
1321
1322
1323
1332
1337
1340
1345
1349
1350
1361
1365
1413
1436
1438
1459
1464
1498
1510
1517
1519
1522
1524
1531
1533
1534
1535
1542
1557
1606
1609
1624
1628
1630
1633
1661
1662
1685
1712
1724
1725
1738
1741
1751
1760
1766
1771
1774
1810
1812
1818
1855
1865
1866
1867
1878
1891
1894
1912
1915
1918
1925
1927
1928
1936
1939
1945
1948
1951
1952
1959
1964
1979
1981
1985
1995
1997
2003
2004
2006
2015
2018
2021
2026
2028
2030
2040
2042
2045
2046
2047
2049
2060
2069
2076
2080
2083
2092
2100
2104
2119
2126
2127
2131
2132
2136
2141
2142
2152
2154
2157
2181
2182
2183
2184
2192
2196
2202
2203
2205
2209
2213
2217
2222
2223
2229
2231
2234
2236
2244
22

In [ ]:
print("length of data in letter or characters")
len(text)

length of data in letter or characters


530397136

In [ ]:
the_chars  = sorted(     list(set(text))     )

vocab_size = len( the_chars )
print(vocab_size)

378


In [ ]:
print("".join(the_chars))

 	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~£®°µ¹ºÁÅÇÉÓÖØÜßàáâãäåæçèéêëìíîïñòóôõöøùúüýāćČčēęěğīıłńőřŞŠšūůŽžſǐɑɪʒʻʼˈː́ΣΫαβεθμπςσϋϔВДИКМОПРСТУФЦавгдезийклмнопрстуфцчщыьюяёֱֶּאבוטכלםנשת״໐ዐᎢᎵᏋᏒᏚ ḥṢṣ–—”•€№Ⅳⅳ→∞≈≡≤≥⋅⌊⌋⌘⎛⎜⎝⎞⎟⎠⎡⎢⎣⎤⎥⎦⎧⎨⎩⎪⎫⎬⎭⎮⎯⎺─│┌┐└├┬┴║╧╱╲╴☺♚♦⟩　〒ノ一东中丸为久京仙代保公内北区千华史司呈和国坡大奥好孫山广彦所技文智有本术李村東松棉水流湖溪田研穂背莞逸郑都限ﬁ﻿０�𐅑🁁🁳🖤


In [ ]:
## Not going to use it now
stoi = { ch:i for i, ch in enumerate(the_chars) } # Character to int
itos = { i:ch for i, ch in enumerate(the_chars) } # Int to char

In [ ]:
import tiktoken

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")
vocab_size = tokenizer.n_vocab #50257

In [ ]:
print(tokenizer.n_vocab)

50257


In [ ]:
encode = lambda s: tokenizer.encode(s)
encodeOld = lambda s: [ stoi[c]          for c in s   ]

In [ ]:
print(encode("bahh")) # Enconding this sheep languaje
print(type(encode("bahh"))) # Enconding this sheep languaje
print(type(encodeOld("bahh")))

[47041, 71]
<class 'list'>
<class 'list'>


In [ ]:
decode = lambda l: tokenizer.decode(l)
decodeOld = lambda l: ''.join(itos[i] for i in l)

In [ ]:
decode([47041, 71])

'bahh'

In [ ]:
data = torch.tensor(   encode(text), dtype=torch.long   ) # Enconde all the book into ints

print( data )

tensor([ 17, 198,  18,  ..., 220, 220, 628])


In [ ]:
print(len(data))
print(data.shape)

194102639
torch.Size([194102639])


In [ ]:
n          = int(   0.9*len(data)   )

train_data = data[:n]
val_data   = data[n:]

In [ ]:
def get_batch(split):
    if split == "train":
        data = train_data
    else:
        data = val_data

    ix = torch.randint(   len(data) - block_size, (batch_size,)   ) # Picks positions randomly as starting point

    x  = torch.stack(    [  data[   i : i+block_size ]     for i in ix ]    ) # This 2 lines create the training and testing sets. Look the notebook *1
    y  = torch.stack(    [  data[ i+1 : i+1+block_size ]   for i in ix ]    ) # This 2 lines create the training and testing sets. Look the notebook *1

    x, y = x.to(device), y.to(device)

    return x, y

# **Get batch**
We define if our data is the training set or the validation set.

We generate as many random numbers as sentences in our block.

In [ ]:
# To estimate performance

@torch.no_grad()    ## for efficient processing
def estimate_loss():
    out = {}
    model.eval()   ## set to no training
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()  ## back to training
    return out

In [ ]:
class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()

        self.key   = nn.Linear(n_embd, head_size, bias=False)  ## [512, 64]
        self.query = nn.Linear(n_embd, head_size, bias=False)  ## [512, 64]
        self.value = nn.Linear(n_embd, head_size, bias=False)  ## [512, 64]

        tril_def = torch.tril( torch.ones(block_size, block_size) )  ## [40, 40]

        self.register_buffer( #Marking it almost as a constant so it doesn't interfere
                  'tril',
                  tril_def
               )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        B, T, E = x.shape   ## [batch_size, 40, 512]

        k = self.key(   x )            ## k = (B, T, 64)
        q = self.query( x )            ## q = (B, T, 64)

        E2 = 64     ## I think this is 64 and not 512
        ## (B, T, E) @ (B, E, T)  -> (B, T, T)
        wei = q @ k.transpose(-2, -1) * E2 ** -0.5

        wei = wei.masked_fill(
                      self.tril[:T, :T] == 0,
                      float('-inf')
        )

        ## (B, T, T)
        wei = F.softmax( wei, dim= -1 )         ## (B, T, T)
        wei = self.dropout(   wei   )

        ## perform weighted aggregation of values

        v   = self.value(  x  )   ## x = (B, 40, E)
        out = wei @ v             ## (B, T, T) @ (B, T, 64) -> (B, T, 64)

        return out

In [ ]:


%%time


import torch
import numpy as np
import torch.nn as nn
import tiktoken

from torch.nn import functional as F

torch.manual_seed(256)
device = 'cuda' if torch.cuda.is_available() else 'cpu' #Cuda is the drivers for Nvidia GPU

block_size        = 40      ## N tokens in sequence -> its not using tiktoken. Tokens are the individual letters
batch_size        = 64
max_iters         = 20000
eval_interval     = 500     # Print what it is doing
learning_rate     = 0.0003  # Control how much we increase or decrease the weight values with the scocastic gradient descent
eval_iters        = 300     # Evaluate every this iterations
vocab_size        = 88      ## 65 -> 26 uppercase, 26 lowercase, digits, simbols...

## every id for a given token is embedded to vector of this size
n_embd            = 512
n_head            = 8         ## 8 attention heads
n_layer           = 6         ## 6 eoncoder layers
dropout           = 0.2       # We will se this later

text = ''

#input_file2 = 'HuckFinn.txt'
input_file2 = '/content/drive/MyDrive/Colab Notebooks/dataFromRFCs.txt' # Full path

with open(input_file2, 'r', encoding='utf-8') as f:
    text = f.read()

print("length of data in letter or characters")
len(text)

list(set(text))

the_chars  = sorted(     list(set(text))     )

tokenizer = tiktoken.get_encoding("gpt2")
vocab_size = tokenizer.n_vocab      ## 65

print(  len(the_chars)  )

print(  ''.join(the_chars)  )

## The printed oputput
## !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz

stoi = { ch:i for i, ch in enumerate(the_chars) } # Character to int
itos = { i:ch for i, ch in enumerate(the_chars) } # Int to char

print( stoi )
print( itos )



encode = lambda s: tokenizer.encode(s)

encode("bahh") # Enconding this sheep languaje

decode = lambda l: tokenizer.decode(l)
decode([55, 54, 61, 61])

data = torch.tensor(   encode(text), dtype=torch.long   ) # Enconde all the book into ints

print( data )

n          = int(   0.9*len(data)   )

train_data = data[:n]
val_data   = data[n:]

def get_batch(split):
    if split == "train":
        data = train_data
    else:
        data = val_data

    ix = torch.randint(   len(data) - block_size, (batch_size,)   ) # Picks positions randomly as starting point

    x  = torch.stack(    [  data[   i : i+block_size ]     for i in ix ]    ) # This 2 lines create the training and testing sets. Look the notebook *1
    y  = torch.stack(    [  data[ i+1 : i+1+block_size ]   for i in ix ]    ) # This 2 lines create the training and testing sets. Look the notebook *1

    x, y = x.to(device), y.to(device)

    return x, y

# THIS IS NOT RELEVANT TO THE HOMEWORKS, JUST CLARIFICATION!!!

temp_batch_size = 4
temp_block_size = 16

## select random starting points for the 4 sentences
ix = torch.randint(
            len(data) - block_size,
            (temp_batch_size,)
)

print( ix ) #Same as in the example because of the seed

for index_temp in ix:
    print(  data[index_temp]  )

x  = torch.stack(
    [ data[   i : i+  temp_block_size ]   for i in ix ]

)

y  = torch.stack(
    [ data[ i+1 : i+1+ temp_block_size ]  for i in ix ]
)

print(x)
print(y)

# To estimate performance

@torch.no_grad()    ## for efficient processing
def estimate_loss():
    out = {}
    model.eval()   ## set to no training
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()  ## back to training
    return out





class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()

        self.key   = nn.Linear(n_embd, head_size, bias=False)  ## [512, 64]
        self.query = nn.Linear(n_embd, head_size, bias=False)  ## [512, 64]
        self.value = nn.Linear(n_embd, head_size, bias=False)  ## [512, 64]

        tril_def = torch.tril( torch.ones(block_size, block_size) )  ## [40, 40]

        self.register_buffer( #Marking it almost as a constant so it doesn't interfere
                  'tril',
                  tril_def
               )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        B, T, E = x.shape   ## [batch_size, 40, 512]

        k = self.key(   x )            ## k = (B, T, 64)
        q = self.query( x )            ## q = (B, T, 64)

        E2 = 64     ## I think this is 64 and not 512
        ## (B, T, E) @ (B, E, T)  -> (B, T, T)
        wei = q @ k.transpose(-2, -1) * E2 ** -0.5

        wei = wei.masked_fill(
                      self.tril[:T, :T] == 0,
                      float('-inf')
        )

        ## (B, T, T)
        wei = F.softmax( wei, dim= -1 )         ## (B, T, T)
        wei = self.dropout(   wei   )

        ## perform weighted aggregation of values

        v   = self.value(  x  )   ## x = (B, 40, E)
        out = wei @ v             ## (B, T, T) @ (B, T, 64) -> (B, T, 64)

        return out

class FeedForward(nn.Module):

    def __init__(self, n_embd):         ## 512

        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),      ## [512, 4*512] -- To give it more room to improve
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),      ## [4*512, 512]
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):    ## (8, 64)
        super().__init__()
        self.heads = nn.ModuleList(  [ Head(head_size) for _ in range(num_heads) ] )
        self.proj  = nn.Linear(n_embd, n_embd)   ## 512, 512
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat(   [ h(x) for h in self.heads ], dim = -1   )
        out = self.proj(  out   )
        out = self.dropout(   out   )
        return out

class Block(nn.Module):

    def __init__(self, n_embd, n_head):     ## (512, 8)
        super().__init__()
        head_size = n_embd // n_head        ## 64 --> 512/8

        self.sa   = MultiHeadAttention(n_head, head_size) # Concatenate the heads

        self.ffwd = FeedForward( n_embd)    ## 512
        self.ln1  = nn.LayerNorm(n_embd)
        self.ln2  = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(     self.ln1(x)      )
        x = x + self.ffwd(   self.ln2(x)      )
        return x

# --------------------------- NN Architectures ---------------------------
# We will use Object Oriented Programming

class GPTModel(nn.Module):
    def __init__(self):
        super().__init__() # We always have this one line

        # ----------------------------------- Semantic and positional Encoding -------------------------------------------------

        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)   ## [65, 512]
        self.pos_emb_table = nn.Embedding(block_size, n_embd)     ## [block, 512]

        # ---------------------------------- End Semantic and positional Encoding-----------------------------------------------


        # ********************************** Create 6 layers of Feed Fordward **********************************

        self.blocks = nn.Sequential(
                *[   Block(n_embd, n_head=n_head) for _ in range(n_layer)    ]
        )

        # ********************************** End Create 6 layers of FF *****************************************


        self.ln_f    = nn.LayerNorm(  n_embd    )


        # ----------------------------------- Last layer -------------------------------------------------

        self.lm_ffw_head = nn.Linear(n_embd, vocab_size)  ## [512, 65] # FFW Layer

        # ----------------------------------- End last layer ---------------------------------------------


    def forward(self, idx, targets=None): #If you dont pass anything on targets, you are in inference mode
        B, T = idx.shape     ## (Batch, 40)
        ## ids and targets are both (B, T) tensors of integers

        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.pos_emb_table(torch.arange(T, device=device))

        x = tok_emb + pos_emb    ## [B, T, E] or [64, 40, 512]

        ## This is the architecture
        x = self.blocks(  x  )   ## (B, T, E)
        x = self.ln_f(    x  )   ## (B, T, E)   ## normalization
        logits = self.lm_ffw_head(x)         ## [B, 40, 88] --> map to the vocabulary

        if targets is None:
            loss = None
        else:
            B, T, E  = logits.shape
            logits  = logits.view( B*T, E)
            targets = targets.view(B*T)
            loss    = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):    ## idx is (B, T)
        for _ in range(max_new_tokens):
            ## crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)    ## ## get preds
            logits = logits[:, -1, :]    ## focus on last one (B, E)
            probs = F.softmax(logits, dim= -1)    ## (B, E) get probs
            idx_next = torch.multinomial(probs, num_samples=1)     ## (B, 1) selected
            idx = torch.cat(  (idx, idx_next), dim=1  )   ## (B, T+1) append sample to running sequence
        return idx

model   = GPTModel()

m       = model.to(device)

optimizer = torch.optim.Adam(  m.parameters(), lr=learning_rate   )





KeyboardInterrupt: 

In [ ]:
## Train the model

for iter in range(max_iters):

    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    ## eval the loss
    logits, loss = m(xb, yb)

    optimizer.zero_grad(set_to_none=True)   ## zero out to prevent them to accumulate
    loss.backward()
    optimizer.step()

NameError: name 'estimate_loss' is not defined

In [ ]:
# Assuming you have already initialized the tokenizer as 'tokenizer'
print(tokenizer.vocab_size)

In [ ]:
new_lst = encode("RFC 5253")

new_np = np.array(  new_lst   )
new_np


# NO TIKTOKEN

In [ ]:
## All code in one

%%time
# Colab training with checkpoints and auto-resume (tiktoken, A100 friendly)

import os, math, time, json
import torch
import torch.nn as nn
from torch.nn import functional as F
import tiktoken

# -------------------- Repro and device --------------------
torch.manual_seed(256)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# -------------------- Paths --------------------
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
INPUT_FILE = '/content/drive/MyDrive/Colab Notebooks/dataFromRFCs.txt'
CKPT_DIR   = '/content/drive/MyDrive/Colab Notebooks/checkpoints_rfc_tiktoken'
os.makedirs(CKPT_DIR, exist_ok=True)

# -------------------- Hyperparams --------------------
# A100 friendly defaults
block_size     = 512
batch_size     = 64
grad_accum     = 2             # effective batch is 64 * 512 * 2 tokens per step
max_iters      = 135_000
eval_interval  = 500
eval_iters     = 200
learning_rate  = 3e-4
dropout        = 0.2

n_embd         = 512
n_head         = 8
n_layer        = 6

# -------------------- Data --------------------
with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    raw_text = f.read()

# tokenizer = tiktoken.get_encoding("gpt2")
# vocab_size = tokenizer.n_vocab  # 50257

# encode = lambda s: tokenizer.encode(s)
# decode = lambda ids: tokenizer.decode(ids)

the_chars  = sorted(list(set(raw_text)))
vocab_size = len(the_chars)

stoi = { ch:i for i, ch in enumerate(the_chars) } # Character to int
itos = { i:ch for i, ch in enumerate(the_chars) } # Int to char

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join(itos[i] for i in l)



ids = encode(raw_text)
data = torch.tensor(ids, dtype=torch.long)

n = int(0.9 * len(data))
train_data = data[:n]
val_data   = data[n:]

def get_batch(split):
    src = train_data if split == "train" else val_data
    ix = torch.randint(len(src) - block_size, (batch_size,))
    x  = torch.stack([src[i : i + block_size] for i in ix])
    y  = torch.stack([src[i + 1 : i + 1 + block_size] for i in ix])
    return x.to(device), y.to(device)

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters, device=device)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss
        out[split] = losses.mean().item()
    model.train()
    return out

# -------------------- Model --------------------
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key   = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        tril = torch.tril(torch.ones(block_size, block_size))
        self.register_buffer('tril', tril)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, E = x.shape
        k = self.key(x)      # (B, T, H)
        q = self.query(x)    # (B, T, H)
        H = q.size(-1)
        wei = q @ k.transpose(-2, -1) * (H ** -0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)    # (B, T, H)
        out = wei @ v        # (B, T, H)
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj  = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa   = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1  = nn.LayerNorm(n_embd)
        self.ln2  = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.pos_emb_table         = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f   = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)  # (B, T, E)
        pos_emb = self.pos_emb_table(torch.arange(T, device=idx.device))  # (T, E)
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)  # (B, T, V)
        loss = None
        if targets is not None:
            B, T, V = logits.shape
            loss = F.cross_entropy(logits.view(B*T, V), targets.view(B*T))
        return logits, loss

    def generate(self, idx, max_new_tokens):
        self.eval()
        with torch.no_grad():
            for _ in range(max_new_tokens):
                idx_cond = idx[:, -block_size:]
                logits, _ = self(idx_cond)
                logits = logits[:, -1, :]
                probs = F.softmax(logits, dim=-1)
                idx_next = torch.multinomial(probs, num_samples=1)
                idx = torch.cat((idx, idx_next), dim=1)
        self.train()
        return idx

# -------------------- Init --------------------
model = GPTModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Mixed precision for speed and memory
use_amp = torch.cuda.is_available()
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

# Optional cosine schedule with warmup
warmup_steps = 2_000
total_steps  = max_iters
def get_lr(step):
    if step < warmup_steps:
        return learning_rate * (step + 1) / warmup_steps
    # cosine decay to 10 percent of base
    progress = (step - warmup_steps) / max(1, total_steps - warmup_steps)
    return 0.1 * learning_rate + 0.9 * learning_rate * 0.5 * (1 + math.cos(math.pi * progress))

# -------------------- Checkpoint helpers --------------------
def save_checkpoint(tag, step, best_val):
    tmp = os.path.join(CKPT_DIR, f"ckpt_{tag}.pt.tmp")
    path = os.path.join(CKPT_DIR, f"ckpt_{tag}.pt")
    payload = {
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "scaler": scaler.state_dict() if use_amp else None,
        "step": step,
        "best_val": best_val,
        "cfg": {
            "block_size": block_size,
            "n_embd": n_embd, "n_head": n_head, "n_layer": n_layer,
            "dropout": dropout, "vocab_size": vocab_size,
            "tokenizer_name": "gpt2",
        },
    }
    torch.save(payload, tmp)
    os.replace(tmp, path)
    return path

def load_checkpoint_if_exists():
    latest = os.path.join(CKPT_DIR, "ckpt_latest.pt")
    if os.path.exists(latest):
        ckpt = torch.load(latest, map_location=device)
        model.load_state_dict(ckpt["model"], strict=True)
        optimizer.load_state_dict(ckpt["optimizer"])
        if use_amp and ckpt.get("scaler") is not None:
            scaler.load_state_dict(ckpt["scaler"])
        step = ckpt.get("step", 0)
        best_val = ckpt.get("best_val", float("inf"))
        print(f"Resumed from step {step}, best_val {best_val:.4f}")
        return step, best_val
    print("No existing checkpoint, starting fresh")
    return 0, float("inf")

# -------------------- Resume if possible --------------------
start_iter, best_val = load_checkpoint_if_exists()

# -------------------- Quick sanity sample before training --------------------
with torch.no_grad():
    ctx = torch.zeros((1,1), dtype=torch.long, device=device)
    sample_ids = model.generate(ctx, max_new_tokens=64)[0].tolist()
print(decode(sample_ids))

# -------------------- Training loop --------------------
save_every = 5_000
t0 = time.time()

model.train()
for iter in range(start_iter, max_iters):

    # LR schedule
    for g in optimizer.param_groups:
        g['lr'] = get_lr(iter)

    if iter % eval_interval == 0:
        losses = estimate_loss(model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, lr {optimizer.param_groups[0]['lr']:.6f}")
        save_checkpoint("latest", iter, best_val)
        if losses['val'] < best_val:
            best_val = losses['val']
            save_checkpoint("best", iter, best_val)
            print("Saved new best checkpoint")

    if iter % save_every == 0 and iter > start_iter:
        save_checkpoint(f"step{iter}", iter, best_val)

    optimizer.zero_grad(set_to_none=True)

    # Gradient accumulation
    total_loss = 0.0
    for _ in range(grad_accum):
        xb, yb = get_batch('train')
        with torch.cuda.amp.autocast(enabled=use_amp):
            logits, loss = model(xb, yb)
            loss = loss / grad_accum
        scaler.scale(loss).backward()
        total_loss += loss.item()

    scaler.step(optimizer)
    scaler.update()

# Final save
save_checkpoint("latest", max_iters, best_val)
print("Training done, total time min:", (time.time() - t0) / 60.0)

Mounted at /content/drive


<timed exec>:189: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.


Resumed from step 135000, best_val 0.7006
 
   3.  Content Format for Word characters......................
step 135000: train loss 0.5889, val loss 0.7072, lr 0.000030


<timed exec>:275: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.


KeyboardInterrupt: 

In [ ]:
import os, torch, tiktoken
import torch.nn as nn
from torch.nn import functional as F
import numpy as np

# Recreate the model class definitions exactly as in training
# If this runs in the same notebook session, you already have them

device = 'cuda' if torch.cuda.is_available() else 'cpu'
CKPT_DIR = '/content/drive/MyDrive/Colab Notebooks/checkpoints_rfc_tiktoken' # Corrected path
best_path = os.path.join(CKPT_DIR, 'ckpt_best.pt')

# Load config and weights
ckpt = torch.load(best_path, map_location=device)
cfg = ckpt['cfg']
block_size = cfg['block_size']
n_embd = cfg['n_embd']
n_head = cfg['n_head']
n_layer = cfg['n_layer']
dropout = cfg['dropout']
# vocab_size is loaded from checkpoint cfg
print("Best validation loss:", ckpt["best_val"])
print("Step:", ckpt["step"])
print()

# ---- No Tokenizer ----
# tokenizer = tiktoken.get_encoding("gpt2")
# vocab_size = tokenizer.n_vocab  # 50257

# encode = lambda s: tokenizer.encode(s)
# decode = lambda ids: tokenizer.decode(ids)

# Load raw_text to build vocabulary for inference
INPUT_FILE = '/content/drive/MyDrive/Colab Notebooks/dataFromRFCs.txt'
with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    raw_text = f.read()

the_chars  = sorted(list(set(raw_text)))
vocab_size = len(the_chars) # Update vocab_size based on raw_text

stoi = { ch:i for i, ch in enumerate(the_chars) } # Character to int
itos = { i:ch for i, ch in enumerate(the_chars) } # Int to char

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join(itos[i] for i in l)


# Rebuild model and load weights
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key   = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        tril = torch.tril(torch.ones(block_size, block_size))
        self.register_buffer('tril', tril)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        B, T, E = x.shape
        k = self.key(x); q = self.query(x); H = q.size(-1)
        wei = q @ k.transpose(-2, -1) * (H ** -0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        return wei @ v

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    def forward(self, x): return self.net(x)

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj  = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa   = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1  = nn.LayerNorm(n_embd)
        self.ln2  = nn.LayerNorm(n_embd)
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.pos_emb_table         = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f   = nn.LayerNorm(n_embd)
        self.lm_head= nn.Linear(n_embd, vocab_size)
    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.pos_emb_table(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        if targets is not None:
            B, T, V = logits.shape
            loss = F.cross_entropy(logits.view(B*T, V), targets.view(B*T))
        return logits, loss
    def generate(self, idx, max_new_tokens):
        self.eval()
        with torch.no_grad():
            for _ in range(max_new_tokens):
                idx_cond = idx[:, -block_size:]
                logits, _ = self(idx_cond)
                logits = logits[:, -1, :]
                probs = torch.softmax(logits, dim=-1)
                idx_next = torch.multinomial(probs, num_samples=1)
                idx = torch.cat((idx, idx_next), dim=1)
        self.train(); return idx

model = GPTModel().to(device)
model.load_state_dict(ckpt['model'], strict=True)
model.eval()

# Generate
with torch.no_grad():
    new_lst = encode("What is IP?")
    new_np = np.array(  new_lst   )
    new_context = torch.tensor(new_np, dtype=torch.long, device=device ).unsqueeze(0) # Added unsqueeze(0)
    out_ids = model.generate(new_context, max_new_tokens=400)[0].tolist()

print(decode(out_ids))

Best validation loss: 0.7005828619003296
Step: 118000

What is IP?
           If the packet is to be PIDF, the reporting point MUST
           resend a CONDSTORE mailbox.
           If the name is OPEN, the reported action MUST be removed.

      *    If the CONDSTORE mailbox is NOT usable and the entry is
           dsult (the carrier issues the CONDSTORE mailbox to be locked
           with any opened file name and changes its own mailbox), then
           whi


In [ ]:
# Generate
with torch.no_grad():
    new_lst = encode("IP address is")
    new_np = np.array(  new_lst   )
    new_context = torch.tensor(new_np, dtype=torch.long, device=device ).unsqueeze(0) # Added unsqueeze(0)
    out_ids = model.generate(new_context, max_new_tokens=1000)[0].tolist()

print(decode(out_ids))

IP address is
   unable to pay a second line for the retry at which only one endpoint
   has a line wreak buffer.

   Since keys are derived from the target, it is possible to determine the
   client's specified sequence number and the server type that may be
   unique among the specified sets of sequence numbers it receives.
   For this reason, the client may know that it wishes to probe the
   issue.  If the string is associated with the specified sequence number
   into the endpoint's sequence, the client SHOULD add a literal literal
   to the user's sequence number for the specified address of the
   client.

   The string in the endpoint's specified Sequence Number is used as a
   connection ID, which MUST be present in the endpoint's sequence
   number into the specified type.

11.4.  CSNP Endpoints

   Although all Sequences Numbers are subject to the issues discussed in
   [RFC4987], it is possible for the client to verify the ciphertext
   value with the CSNP Endpoint ID.  Ind